source of all code fragments: https://github.com/jessevig/bertviz

In [14]:
! pip install bertviz transformers

In [15]:
from bertviz import model_view, head_view
from transformers import AutoTokenizer, AutoModel, utils

utils.logging.set_verbosity_error()  # Suppress standard warnings

In [16]:
tokenizer = AutoTokenizer.from_pretrained("botryan96/GeoBERT")
try:
  model = AutoModel.from_pretrained("botryan96/GeoBERT", output_attentions=True)
except:
  model = AutoModel.from_pretrained('botryan96/GeoBERT', from_tf = True, output_attentions=True)

inputs = tokenizer.encode("""ExxonMobil plans to drill an exploratory well in the North Sea.""", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0])
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [ ]:
model_view(attention, tokens)

In [13]:
import torch
method = "max"
# Assuming you have already tokenized and obtained attention weights as in your previous code

# Choose the attention weights from the last layer
# Assuming a batch size of 1, we'll take the 0th index
last_layer_attention = attention[-1][0]

# Aggregate subtokens and attention weights
new_tokens = []
new_attention = []

i = 0
while i < len(tokens):
    token = tokens[i]

    if not token.startswith("##"):
        # This is a main token
        new_tokens.append(token)

        # Check if the next tokens are subtokens
        subtoken_attention = [last_layer_attention[:, i, :]]
        while (i+1) < len(tokens) and tokens[i+1].startswith("##"):
            i += 1
            subtoken_attention.append(last_layer_attention[:, i, :])

        # Aggregate the attention weights
        if method == "average":
            aggregated_attention = torch.mean(torch.stack(subtoken_attention), dim=0)
        elif method == "max":
            aggregated_attention, _ = torch.max(torch.stack(subtoken_attention), dim=0)

        new_attention.append(aggregated_attention)
    i += 1

# Convert the list of aggregated attention tensors back to the expected shape
new_attention = torch.stack(new_attention, dim=1)

# Reshape the new_attention tensor to fit the expected shape
new_attention_reshaped = new_attention.unsqueeze(0).unsqueeze(0)

# Now use BertViz with the reshaped attention tensor and new tokens
head_view(new_attention_reshaped, new_tokens)



ValueError: ignored